In [ ]:
#랭체인 설치
!pip install langchain
#langchain이 openai를 활용해서 사용하게 하기위한 라이브러리
!pip install langchain_openai
#langchain 자체 라이브러리
!pip install openai

In [ ]:
import os
import openai

#이전의 방식은 소스코드를 내가 업로드 했을 떄, 다른 사람들이 내 키를 볼 수 있음
#'이름'==userdata.get(이름),값=내 open_api_key
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPEN_AI_KEY')

openai.api_key=os.getenv('OPENAI_API_KEY')

##체인

-  | (체인)
- 여러개의 작업을 연결하여 수행
- 여러번의 정의가 필요하지 않음 -> 한번에 엮어서 순차적으로 수행가능

In [ ]:
#message.choice[0].content X-> 내용만 가져올 수 있게 도와주는 함수
from langchain_core.output_parsers import StrOutputParser

#대화형 프롬프트를 정의
from langchain_core.prompts import ChatPromptTemplate

#openai와 대화할 수 있게 하는 클래스
from langchain_openai import ChatOpenAI

In [ ]:
#채팅 모델 정의(openai에 질의응답을 전달할 수 있도록 함)
chat_model=ChatOpenAI()

#챗프롬프트템플릿
#동일한 내용에 대해 전달할 때 {topic}만 변경하면 됨
chat_prompt_template=ChatPromptTemplate.from_template('tell me a short joke about {topic}')

#콘텐츠만 추출하도록 도와주는
output_parser=StrOutputParser()

#response=chat_model.invoke(messages)
#print(response.content)

#체인정의
#내가 물어볼 질문의 구조를 미리 짜놓음 -> 모델에 집어넣기 -> 내용만 추출
chain=chat_prompt_template| chat_model| output_parser

chain.invoke({'topic':'programmer'})

In [ ]:
#챗프롬프트템플릿
#동일한 내용에 대애 전달할 때 {topic}만 변경하면 됨
chat_prompt_template=ChatPromptTemplate.from_template('please explain the {city} history.')

chain=chat_prompt_template |ChatOpenAI() |StrOutputParser()
chain.invoke({'city':'seoul'})

## Runnable 객체

- RunnablePassthrough: 입력값을 -출력값으로 전달
- RunnableParaller: 여러개의 Runnable을 병렬로 실행
- RunnableLambda : 람다 함수나, 임의의 함수를 사용할 수 있게 감싸줌

In [ ]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel,RunnableLambda

In [ ]:
chat_prompt_template=ChatPromptTemplate.from_template("what's your favortite {thing}")

chat_model=ChatOpenAI()
output_parser=StrOutputParser()

#체인의 순서가 바뀌어도 되는가?no
chain={'thing':RunnablePassthrough()}|chat_prompt_template | chat_model | output_parser

In [ ]:
chain.invoke('color')

In [ ]:
chain.invoke('flower')

In [ ]:
chat_prompt_template=ChatPromptTemplate.from_template("what's your favortite {thing}")

chat_model=ChatOpenAI()
output_parser=StrOutputParser()

#체인의 순서가 바뀌어도 되는가?no
chain=RunnablePassthrough()|chat_prompt_template | chat_model | output_parser
chain.invoke({'thing':'city'})

###러너블 객체 활용하기

1. 객체 방식으로 접근하기

    #### dunder메서드(매직메서드->__)
    chain=a.__or__(b)
    __init__

2. 파이프라인 문법으로 접근하기
    - chain=a | b


In [ ]:
#Runnable이라는 객체 Runnable()
#내가 만든 함수를 Runnable 체인 사이에 끼우고 싶어!
class Runnable:
    #class가 처음 정의될 때
    #runnable을 만들때, 함수를 하나 전달해서 함수를 'runnable'객체로 만드는 과정
    #굳이 왜? 체인에 포함시키고 싶어서
    def __init__(self,func):
        self.func=func

    #chain 문법을 사용하기 위해
    #함수a | 함수b
    def __or__(self,other):

        #*args=어떤 파라미터를 전해주던지, 받아들일 수 있게
        #어떤 파라미터를 얼마나 넣을지 모름
        def chained_func(*args,**kwars):
            return other(self.func(*args,**kwars))

        return Runnable(chained_func)

    #호출 가능하도록
    def __call__(self,*args,**kwars):
        return self.func(*args,**kwars)

####*args,**kwars 설명


In [ ]:
#*args,**kwars 설명
def add(*args):
    for a in args:
        print(a)

In [ ]:
add(1,2,'string') #모든 걸 다 받아줌/뭘, 얼마나 줄지 모름

In [ ]:
#키워드 알규먼트
#키워드 몇개에 집어넣을거임
def pr(**kwargs):
    for k,v in kwargs.items():
        print(f"{k}={v}")

In [ ]:
pr(a=1)

In [ ]:
def add_five(x):
    return x+5

def multiply_by_two(x):
    return x*2

###Runnable 사용

In [ ]:
add_five=Runnable(add_five)
multiply_by_two=Runnable(multiply_by_two)

In [ ]:
#|으로 연결해서 하나의 함수로 생성
chain=add_five | multiply_by_two
chain(3)

In [ ]:
chain=add_five.__or__(multiply_by_two)
chain(3)

In [ ]:
#파이프라인, or은 결과가 동일함

###RunnablePassthrough


In [ ]:
from langchain_core.runnables import RunnablePassthrough

templates='Please generate three sentences in a dialougue in {lang} on the topic {topic}'

#내가 묻고싶은 뼈대구성
chat_prompt_template=ChatPromptTemplate.from_template(templates)

#뼈대를 모델에 넣고 내용만 추출하게끔
chain=chat_prompt_template | chat_model | StrOutputParser()

#뼈대에 내가 묻고싶은 거 넣기
output=chain.invoke({"lang":"french","topic":"vacance"})
print(output)

In [ ]:
#_ : 매개변수가 뭐가 들어오던지 'french'를 내보내
def return_lang(_):
    return 'french'

def return_topic(_):
    return 'olympic'

In [ ]:
#하나만 전달해줄고 나머지는 함수로
chain = RunnablePassthrough.assign(lang = return_lang)| chat_prompt_template | chat_model | StrOutputParser()

output = chain.invoke({"topic" : "summer"})
print(output)